In [1]:
import torch
from torch import nn

import numpy as np
from tqdm import tqdm

import sys
import datetime

sys.path.append('../code')
from dataset import get_data, MaskedDataset, make_vocab

from transformers import (
    AdamW, get_linear_schedule_with_warmup
)

from models import MaskedRDModel

In [2]:
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')

Using cache found in /Users/axie/.cache/torch/hub/huggingface_pytorch-transformers_master


In [6]:
model = MaskedRDModel.from_pretrained('bert-base-uncased')
model.set_mask_size(5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing MaskedRDModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing MaskedRDModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MaskedRDModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
d = get_data('../wantwords-english-baseline/data', word2vec=False)

Loading data...
Training data: 675715 word-def pairs
Dev data: 75873 word-def pairs
Test data: 1200 word-def pairs


In [4]:
train_data, train_data_def, dev_data, test_data_seen, \
    test_data_unseen, test_data_desc = d

In [5]:
target_matrix, target2idx, idx2target = make_vocab(d, tokenizer, mask_size=5)
train_dataset = MaskedDataset(train_data, tokenizer, target2idx, target_matrix, mask_size=5)

In [40]:
# target2idx maps target words to indices
# target_matrix maps target indices to bpe sequences, padded/truncated to mask_size
target2idx['book'], target_matrix[target2idx['book']]

(16187, tensor([2338,  103,  103,  103,  103]))

In [10]:
x, y = train_dataset[100]
x = x.unsqueeze(0)
attention_mask = x > 0

batchX, batchY = train_dataset.collate_fn([train_dataset[i] for i in range(40, 42)])

In [11]:
batchX

tensor([[  101,   103,   103,   103,   103,   103,   102,  2008,  2029,  3310,
          2013, 19610,  2361,  1999,  1996,  2832,  1997, 11300, 18809,  2290],
        [  101,   103,   103,   103,   103,   103,   102,  2000,  3395,  2000,
          2019,  2552,  1997,  2061,  9527,  2100,  2926, 20951,     0,     0]])

In [13]:
target_matrix = torch.stack(target_matrix)

In [33]:
out = model(input_ids=batchX, attention_mask=batchX > 0, target_matrix=target_matrix)

In [35]:
out.shape

torch.Size([1, 5, 50477])

In [34]:
torch.argmax(out.sum(dim=1), dim=-1)

tensor([4128])

In [29]:
for w in target2idx:
    if target2idx[w] == 23413:
        break
print(w)

unicyclist


In [32]:
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(x[0].numpy()))

'[CLS] [MASK] [MASK] [MASK] [MASK] [MASK] [SEP] characterized by three colors in a specific sense having the three fundamental color sensations of red green and purple as the normal eye in distinction from a color blind eye which can perceive only two of the fundamental colors'

In [ ]:
class MaskedRD(nn.Module):
    def __init__(self, )